



<div class="row">
  <div class="column">
    <img src="../../../img/Atr_logo.png" width="250">
  </div>
</div>





# Gas turbine L0 model


The L0 gas turbine model consists of simple empirical relations capable of determining the main engine performance parameters as a function of atmospheric conditions. It uses Eqs. (4) and (5) to estimate the available power and the specific fuel consumption SFC (lb/hr/shp). 

<div class="row">
  <div class="column">
    <img src="../../../img/GT_L0_eqs.png" width="400">
  </div>
</div>


Equation (4) calculates the available power taking into account altitude effects and the velocity. The maximal power that a gas turbine engine is able to generate is affected by the altitude due to the change in air density. Moreover, due to pressure recovery in the engine air intake, the power increases with flight velocity. The specific fuel consumption of the engine is influenced by engine size, engine throttle and atmospheric conditions. The SFC is modeled as relative increase of the specific fuel consumption at take-off, taken from the PW127 engine datasheet. Throttle setting and atmospheric conditions are taken into account with the corrected part-load factor p_corrected as given in Eqs. (6) and (7).  θ and δ are respectively the ambient pressure ratio and the temperature ratio.
<div class="row">
  <div class="column">
    <img src="../../../img/GT_L0_eqs2.png" width="400">
  </div>
</div>

In [1]:
#Run this cell to import the needed modules/packages  N.B: Double click on the left to expand the window
'''
FOR THE EXTENSION OF CELL COLLAPSING:
pip install nbextension-cellfolding
jupyter nbextension install --py cellfolding --user
jupyter nbextension enable --py cellfolding --user 
'''
from fastoad.utils.physics import Atmosphere
from scipy import constants

import numpy as np
import pandas as pd
import os.path as pth
import matplotlib.pyplot as plt
from fastoad.io import VariableIO 
from fastoad.utils.postprocessing import VariableViewer

import sys
sys.path.append('C:\\Users\\LA202059\\Desktop\\RHEA\\rhea\\models\\propulsion\\fuel_engine\\turboprop_engine')
sys.path.append('C:\\Users\\LA202059\\Desktop\\RHEA\\rhea')
sys.path.append('C:\\Users\\LA202059\\Desktop\\RHEA')
from TP_engine_L0 import TPEngine_L0
from fastoad.utils.physics import Atmosphere
from openmdao.core.explicitcomponent import ExplicitComponent

#sys.path


Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


Import and modify (if needed) RHEA output file of ATR72 model 

In [2]:
INPUT_FILE = 'ATR72_inputs.xml'
vv = VariableViewer()
vv.load(INPUT_FILE)
vv.display()

In [3]:
vv.load(INPUT_FILE)
inputs=vv.get_variables()

In [4]:
#Definition of input variable
RTO_power=                np.asarray(inputs["data:propulsion:RTO_power"].value)
Design_Thermo_Power=      np.asarray(inputs["data:propulsion:Design_Thermo_Power"].value)
Power_Offtake=            np.asarray(inputs["data:propulsion:Power_Offtake"].value)
gearbox_eta=              np.asarray(inputs["data:propulsion:gearbox_eta"].value)


In [5]:
phase=np.array([3])   #TO = 1,MCL = 2,MCR = 3,FID = 4, RTO = 8'''
mach=0.15        
altitude = 20000*constants.foot
out_power_rate=1



psfc_0 =TPEngine_L0(RTO_power,
                    Design_Thermo_Power,
                    Power_Offtake,
                    gearbox_eta
                    ).sfc_at_max_power() #lb/hp/hr 

atmosphere = Atmosphere(altitude, altitude_in_feet=False)
max_shaft_power,max_thermo_power,gearbox_limit_power    = TPEngine_L0(  RTO_power,
                                                                                Design_Thermo_Power,
                                                                                Power_Offtake,
                                                                                gearbox_eta
                                                                               ).max_power(atmosphere, mach, phase)
out_power=max_shaft_power*out_power_rate

psfc = psfc_0 * TPEngine_L0(RTO_power,
            Design_Thermo_Power,
            Power_Offtake,
            gearbox_eta
            ).sfc_ratio(atmosphere, mach, out_power_rate, out_power) #lb/hp/hr 

print(altitude/constants.foot,mach,float(max_shaft_power)/constants.hp, psfc*constants.lb)




TypeError: __init__() missing 9 required positional arguments: 'd_prop', 'k_gb_RTO', 'k_gb_NTO', 'k_gb_MCL', 'k_gb_MCT', 'k_gb_MCR', 'k_gb_FID', 'k_psfc', and 'k_prop'

## 4. Validation

The reference file "PW127F-M-N_complete_datapack.xlsx" was used to validate the results. For the ATR72 the following sheets were used: TO_M, RTO_M, MCL, MCR, FID, CR_variable.

In [ ]:
ref_TO = pd.read_csv('../../../validation/PW127M_ref/PW127M_deck_SHP_TO2.txt',sep="\t")
ref_MCL = pd.read_csv('../../../validation/PW127M_ref/PW127M_deck_SHP_MCL.txt',sep="\t")
ref_CR = pd.read_csv('../../../validation/PW127M_ref/PW127M_deck_SHP_CRZ.txt',sep="\t")
ref_RTO = pd.read_csv('../../../validation/PW127M_ref/PW127M_deck_SHP_RTO2.txt',sep="\t")
ref_FID = pd.read_csv('../../../validation/PW127M_ref/PW127M_deck_SHP_FID.txt',sep="\t")


In [ ]:
phase=np.array([3])   #TO = 1,MCL = 2,MCR = 3,FID = 4, RTO = 8'''
speeds=[0.01,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]        
altitudes = [0,2500*constants.foot,5000*constants.foot,10000*constants.foot,15000*constants.foot,20000*constants.foot,25000.*constants.foot]#,30000.*constants.foot]


psfc_0 =TPEngine_L0(RTO_power,
                    Design_Thermo_Power,
                    Power_Offtake,
                    gearbox_eta
                    ).sfc_at_max_power() #lb/hp/hr 



def create_df(phase):
    df= pd.DataFrame()
    for mach in speeds:
        for altitude in altitudes:
            atmosphere = Atmosphere(altitude, altitude_in_feet=False)
            max_shaft_power,max_thermo_power,gearbox_limit_power    = TPEngine_L0(  RTO_power,
                                                                                            Design_Thermo_Power,
                                                                                            Power_Offtake,
                                                                                            gearbox_eta).max_power(atmosphere, mach, phase)
            out_power_rate=1                                                                               
            out_power=max_shaft_power
            psfc = psfc_0 * TPEngine_L0(RTO_power,
                        Design_Thermo_Power,
                        Power_Offtake,
                        gearbox_eta
                        ).sfc_ratio(atmosphere, mach, out_power_rate, out_power) #lb/hp/hr 
            df = df.append({'Alt_ft':altitude/constants.foot,'Mach':mach, 'SHP':float(max_shaft_power)/constants.hp,'psfc':psfc*constants.lb}, ignore_index=True)
    return df   

rhea_TO=create_df(1)
rhea_MCL=create_df(2)
rhea_MCR=create_df(3)
rhea_FID=create_df(4)
rhea_RTO=create_df(8)



In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
speeds=[0.01,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5]        
altitudes = [0,2500,5000,10000,20000,25000.]
psfc_pred=[]
psfc_true=[]
shp_pred=[]
shp_true=[]
for alt in altitudes:
    for mach in speeds:
        SHP_pred=rhea_MCL.loc[rhea_MCL['Alt_ft']==alt, ['SHP','Mach']].loc[rhea_MCL['Mach']==mach, ['SHP']].values[0][0]
        PSFC_pred=rhea_MCL.loc[rhea_MCL['Alt_ft']==alt, ['psfc','Mach']].loc[rhea_MCL['Mach']==mach, ['psfc']].values[0][0]
        if mach==0.01:
            mach=0
        SHP_true=ref_MCL.loc[ref_MCL['Altitude']==alt, ['SHP','Mach']].loc[ref_MCL['Mach']==mach, ['SHP']].values[0][0]
        PSFC_true=ref_MCL.loc[ref_MCL['Altitude']==alt, ['PSFC','Mach']].loc[ref_MCL['Mach']==mach, ['PSFC']].values[0][0]     
        
        psfc_pred.append(PSFC_pred)
        psfc_true.append(PSFC_true)
        shp_pred.append(SHP_pred)
        shp_true.append(SHP_true)

    



fig = plt.figure()
plt.plot(shp_true, shp_true, '-', label='$SHP_{true}$')
plt.plot(shp_true, shp_pred, 'r.', label='$\hat{SHP}$')
   
plt.xlabel('$SHP_{true}$')
plt.ylabel('$\hat{SHP}$')

plt.legend(loc='upper left')
print('Mean squared error SHP', mean_squared_error(shp_true,shp_pred)) 
print('Mean absolute error SHP', mean_absolute_error(shp_true,shp_pred)) 

fig = plt.figure()
plt.plot(psfc_true, psfc_true, '-', label='$PSFC_{true}$')
plt.plot(psfc_true, psfc_pred, 'r.', label='$\hat{PSFC}$')
   
plt.xlabel('$PSFC_{true}$')
plt.ylabel('$\hat{PSFC}$')

plt.legend(loc='upper left')
print('Mean squared error PSFC', mean_squared_error(psfc_true,psfc_pred)) 
print('Mean absolute error PSFC', mean_absolute_error(psfc_true,psfc_pred)) 